 # CSV TO MS SQL SERVER

     # pyodbc is an open source Python module that makes accessing ODBC databases simple. It implements the DB API 2.0       specification but is packed with even more Pythonic convenience. I am using MS SQL server, to install pyodbc just go to cmd      and type 'pip install pyodbc' then it's ready to be used.
     https://docs.microsoft.com/en-us/sql/connect/python/pyodbc/python-sql-driver-pyodbc?view=sql-server-ver15      

In [138]:
import numpy as np
import pandas as pd
import pyodbc      

In [139]:
file_path='C:/Users/Hp/Desktop/From_CSV_to_MSSQL_server/sales.csv'
df=pd.read_csv(file_path) # to look at it what kinda data we have in it


In [140]:
df.tail() #this will let you get the top 5 rows from the Dataframe

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.0,0.0,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.0,0.0,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.0,0.0,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.0,0.0,0.01
16597,16600,Spirits & Spells,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.0,0.0,0.01


In [141]:
df.columns=[x.lower().replace("?","_").replace("#","").replace("*","")\
            for x in df.columns]
# I am going to change the columns' names to lower case and get rid of other things like ? # *

In [142]:
df.isnull().values.any() #to check to see if there are any null or NaN data types.


True

In [143]:
df.isnull().sum() # this is a control process to check to see if there is any missing value as null or NaN

rank              0
name              0
platform          0
year            271
genre             0
publisher        58
na_sales          0
eu_sales          0
jp_sales          0
other_sales       0
global_sales      0
dtype: int64

In [144]:
df.isnull().sum().sum() #this is showing you total number of missing value, if you keep going like that and not 
#delete and get rid of any missing value you get an error while inserting any value to sql server

329

In [147]:
#lets get rid of all of them 
df=df.dropna(how='any') # mission accomplished

In [148]:
df.dtypes # checked to see what kinda datA types I have in the DataFrame cause; I need to convert them into right data type

rank              int64
name             object
platform         object
year            float64
genre            object
publisher        object
na_sales        float64
eu_sales        float64
jp_sales        float64
other_sales     float64
global_sales    float64
dtype: object

In [149]:
df["year"] = df["year"].astype(int) # I converted the data type of df["year"] from float to int

In [ ]:
#as I can see our data is ready to be inserted in to MS sql server before doing it we need to make a connection from python to
#ms sql server so lets jump into it

In [150]:
check_for_nan = df[df.isna().any(axis=1)]
check_for_nan  # yuo can check like this too, we dont have any missing or null or NaN value

,rank,name,platform,year,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales


In [130]:
servername='database-redyellow.cloc5pbzda4k.us-east-2.rds.amazonaws.com'
database='BikeStores'
username='admin'
password='Melo.539933'
conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+servername+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()
print("connected!!")  # this is all it takes to connect to our ms sql database I am using AWS.

connected!!


In [131]:
for x in df.columns:
    print(x,end=",") #you can fetch all column names from here easily
    


rank,name,platform,year,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales,

In [106]:
for x in range(len(df.columns)):
    print("?",end=",") 

#to see how many column I will add into db  I will call these 11 question marks as 'values'   
    

?,?,?,?,?,?,?,?,?,?,?,

In [129]:
cursor.execute("drop table if exists sold")
conn.commit() #to chack to see if there is a table named 'sold' in db

In [132]:
convert={
       "int64":"int",
       "object":"nvarchar(250)",
       "float64":"float",
       "int32":"int"
}

In [133]:
line = ", ".join("{} {}".format(x, y) for (x, y) in zip(df.columns, df.dtypes.replace(convert)))
line #this is the columns of the table with data types that ı want to create in my db 

'rank int, name nvarchar(250), platform nvarchar(250), year int, genre nvarchar(250), publisher nvarchar(250), na_sales float, eu_sales float, jp_sales float, other_sales float, global_sales float'

In [134]:

#so we can create a table named 'sold'
cursor.execute("create table sold (%s)" % (line))
conn.commit()


In [136]:
names='rank,name,platform,year,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales'
values='?,?,?,?,?,?,?,?,?,?,?'
cursor.fast_executemany = True
cursor.executemany("insert into sold (%s) values (%s)" % (names,values),df.values.tolist())
print('{} rows inserted to the sold table'.format(len(df)))
cursor.commit()
conn.close()

# it's really so fast that we just inserted 16291 rows within just 10 secs, we cant do it with just this codes belove
#----------------------------------------------------------------------------------------------------------
# for row in x.itertuples():
#       cursor.execute("insert into sales (rank,name,platform,year,genre,pu_lisher,na_sales,eu_sales,\
#                     jp_sales,other_sales,glo_al_sales) values(?,?,?,?,?,?,?,?,?,?,?)",row.rank,row.name,\
#                     row.platform,row.year,row.genre,row.pu_lisher,row.na_sales,row.eu_sales,row.jp_sales,\
#                     row.other_sales,row.glo_al_sales)
# conn.commit()
# cursor.close()
#----------------------------------------------------- its gonna take a lot time to insert them into db with this code structure

16291 rows inserted to the sold table


In [123]:
#you can easily get all data from sql using this simple code structure belove
# we need to make a connection again because we attempted to close the connection with what we did above( cursor.close())

In [115]:
#lets remake a connection
servername='database-redyellow.cloc5pbzda4k.us-east-2.rds.amazonaws.com'
database='BikeStores'
username='admin'
password='Melo.539933'
conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+servername+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()
print("connected!!")

connected!!


In [116]:
#we can fetch all data again from db typing just one code row
df1=pd.read_sql_query('SELECT * FROM sold',conn) 

In [121]:
#then we converted the data that we fetched from db from DataFrame to CSV
df1.to_csv('C:/Users/Hp/Desktop/From_CSV_to_MSSQL_server/converted_from_df_to_CSV.csv')

In [122]:
#to convert it to excel you need install openpyxl typing 'pip install openpyxl' in cmd, if you have it skip it 
df1.to_excel("C:/Users/Hp/Desktop/From_CSV_to_MSSQL_server/converted_from_df_to_excel.xlsx")
